In [4]:
import requests
from requests import Session
import pandas as pd

# Public API

In [5]:
class MarketData:
    # https://docs.kucoin.com/ - API Documentaion

    def __init__(self):
        self.apiurl = "https://api.kucoin.com"  # base url for kucoin
        self.session = Session()

    # returns all tickers
    def getAllTickers(self):
        url = self.apiurl + "/api/v1/market/allTickers"
        r = self.session.get(url)
        data = r.json()["data"]["ticker"]
        df = pd.DataFrame.from_dict(data, orient="columns")
        return df.loc[
            :,
            [
                "symbolName",
                "buy",
                "sell",
                "changeRate",
                "changePrice",
                "high",
                "low",
                "vol",
                "last",
                "averagePrice",
            ],
        ]

    # returns a single trading pair ex. 'BTC-USDT'
    def getTicker(self, symbol):
        url = self.apiurl + "/api/v1/market/orderbook/level1"
        parameters = {"symbol": symbol}
        r = self.session.get(url, params=parameters)
        data = r.json()["data"]
        # dictionary to dataframe
        df = pd.DataFrame.from_dict(data, orient="index", columns=[symbol])
        return df
    
    def get24hrStats(self, symbol):
        url = self.apiurl + "/api/v1/market/stats"
        parameters = {"symbol": symbol}
        r = self.session.get(url, params=parameters)
        data = r.json()["data"]
        # dictionary to dataframe
        df = pd.DataFrame.from_dict(data, orient="index", columns=[symbol])
        return df

    # returns the order book for a trading pair
    def getPartOrderBook(self, symbol, amount):  # does not required API KEY
        # amount can be 20 pieces of data or 100 pieces of data
        url = self.apiurl + "/api/v1/market/orderbook/level2_" + amount
        parameters = {"symbol": symbol, "amount": amount}
        r = self.session.get(url, params=parameters)
        bids = r.json()["data"]["bids"]
        asks = r.json()["data"]["asks"]
        dfBids = pd.DataFrame.from_dict(bids, orient="columns")
        dfAsks = pd.DataFrame.from_dict(asks, orient="columns")
        dfAsks.columns = ["", "Asks"]
        dfBids.columns = ["", "Bids"]
        return dfBids, dfAsks

    # returns the fiat price of a symbol
    def getFiatPrice(self, symbol):
        url = self.apiurl + "/api/v1/prices"
        parameters = {"symbol": symbol}
        r = self.session.get(url, params=parameters)
        data = r.json()["data"][symbol]
        return data

In [6]:
kucoinAPI = MarketData()

In [7]:
kucoinAPI.getAllTickers()

,symbolName,buy,sell,changeRate,changePrice,high,low,vol,last,averagePrice
0,NKN-USDT,0.0831,0.0837,-0.1074,-0.0101,0.0959,0.081,453167.2283061,0.0839,0.09500552
1,LOOM-BTC,0.0000017,0.00000171,-0.0228,-0.00000004,0.00000182,0.00000168,766689.39387906,0.00000171,0.00000173
2,JUP-ETH,0.00000241,0.00000243,-0.0966,-0.00000026,0.00000269,0.00000228,10679388.54196747,0.00000243,0.00000302
3,CUSD-USDT,0.9984,0.9994,-0.0006,-0.0006,0.9999,0.9964,233309.8819412,0.9984,0.99651441
4,LTC3L-USDT,0.007755,0.007801,-0.185,-0.001805,0.010179,0.006052,4438381.42257545,0.007949,0.01072637
...,...,...,...,...,...,...,...,...,...,...
1229,FRR-USDT,0.0144,0.0145,-0.2615,-0.0051,0.027,0.014,62940091.56072818,0.0144,0.02051925
1230,DMTR-USDT,0.0588,0.0589,-0.2008,-0.0148,0.0756,0.055,3734880.53413164,0.0589,0.07467701
1231,GMB-BTC,0.0000000176,0.0000000183,0.0643,0.0000000011,0.0000000215,0.0000000163,15622726.38767027,0.0000000182,0.00000001
1232,TRIBE-USDT,0.2429,0.2433,-0.0244,-0.0061,0.2508,0.2429,11296.31640047,0.2429,0.26305154


In [8]:
kucoinAPI.getTicker('ETH-USDT')

,ETH-USDT
time,1653595826748
sequence,1628370824545
price,1814.9
size,0.0029754
bestBid,1815.07
bestBidSize,4.86526
bestAsk,1815.08
bestAskSize,2.5193449


In [12]:
kucoinAPI.getPartOrderBook('ETH-USDT', '20')

(                   Bids
 0   1817.93   2.1738179
 1    1817.9   4.5854542
 2   1817.82   7.3784901
 3   1817.81      0.1375
 4    1817.8  23.4796024
 5   1817.79   4.6059112
 6   1817.75      0.0018
 7   1817.73   6.1985052
 8   1817.72   6.1617285
 9   1817.71   2.4007925
 10  1817.61      0.0096
 11  1817.58   0.1751768
 12  1817.57     1.02525
 13  1817.56     1.02525
 14  1817.54   6.0844973
 15  1817.45     1.02525
 16  1817.43    0.225447
 17  1817.42   1.1555765
 18  1817.38   0.5546771
 19   1817.3         8.5,
                   Asks
 0   1817.94      0.002
 1   1817.95     0.0395
 2   1817.96    9.52525
 3   1817.99        8.5
 4   1818.03    1.02525
 5   1818.18  1.3767588
 6   1818.31    4.17525
 7   1818.34     0.0094
 8   1818.36  0.5542797
 9   1818.37        0.8
 10  1818.38       0.51
 11  1818.42        4.8
 12  1818.44        1.6
 13  1818.45        0.5
 14  1818.49        3.2
 15  1818.53     0.0099
 16  1818.54  0.2063827
 17  1818.56  4.7954096
 18  1818.63  2.32

In [13]:
kucoinAPI.getFiatPrice('ICP')

'7.30195877'

In [57]:
kucoinAPI.get24hrStats('ICP-USDT')

,ICP-USDT
time,1653546776262
symbol,ICP-USDT
buy,7.47
sell,7.48
changeRate,-0.0386
changePrice,-0.3
high,7.92
low,7.46
vol,141401.73479422
volValue,1098081.3871179039
